# Select strain_data as data directory

In [1]:
import os
print(os.getcwd())
os.chdir('/mnt/ssd/git/dynamic-cmr-models/')
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()

from ipywidgets import interact_manual
import ipywidgets as widgets
from src.utils.Evaluation import plot_strain_per_time
import pandas as pd
import glob
###Choose data-directory
from ipyfilechooser import FileChooser
import os
from IPython.display import display
path_chooser = FileChooser(os.path.join(os.getcwd(), 'strain_predictions', "strain_data"))
load_controls_checkbox = widgets.Checkbox(
    value=False,
    description='choose data directory',
    disabled=False,
    indent=False)
display(path_chooser, load_controls_checkbox)

/mnt/ssd/git/strain_predictions
search for root_dir and set working directory
Working directory set to: /mnt/ssd/git/dynamic-cmr-models


/home/sven/anaconda3/envs/dcmr/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


FileChooser(path='/mnt/ssd/git/dynamic-cmr-models/strain_predictions/strain_data', filename='', title='HTML(va…

Checkbox(value=False, description='choose data directory', indent=False)

# Merge strain dataframes with corresponding metadata dataframe

In [2]:
from strain_predictions.helperfunctions import merge_strain_and_tofmeta
data_root = glob.glob(os.path.join(path_chooser.selected,"**"), recursive=True)
@interact_manual
def merge(gcn_ed2p_path=data_root,
          gcn_p2p_path=data_root,
          gcn_metadata_path=data_root,
          indctr_ed2p_path=data_root,
          indctr_p2p_path=data_root,
          indctr_metadata_path=data_root):
    
    import pandas as pd
    
    gcn_strain_ed2p = pd.read_csv(gcn_ed2p_path)
    gcn_strain_p2p = pd.read_csv(gcn_p2p_path)
    gcn_metadata = pd.read_csv(gcn_metadata_path)
    
    
    indicator_strain_ed2p = pd.read_csv(indctr_ed2p_path)
    indicator_strain_p2p = pd.read_csv(indctr_p2p_path)
    indicator_metadata = pd.read_csv(indctr_metadata_path)

    dfs = [[gcn_strain_ed2p,gcn_strain_p2p,gcn_metadata],[indicator_strain_ed2p,indicator_strain_p2p,indicator_metadata]]
    
    for dataset_dfs in dfs:
        merge_strain_and_tofmeta(strain_ed2p_df=dataset_dfs[0],
                                 strain_p2p_df=dataset_dfs[1],
                                 metadata_df=dataset_dfs[2]
                                )
        
        print("New dataframes are saved in working directory.")
        
    from strain_predictions.helperfunctions import standardize_strain_df
    
    ## I checked the excel sheets manually and found that the only metadata coloumns that intersect are gender and patient ID.
    standardize_strain_df(drop_unamed_columns=True) ###Standardizing columns

ModuleNotFoundError: No module named 'strain_predictions'

# Display violine plots and statistical significance for deceased vs non-deceased

In [5]:
@interact_manual
def play(tof_indicator=True, tof_gcn=False, per_aha=False):
    
    if tof_indicator is True and tof_gcn is False:
        strain_ed2p = pd.read_csv(os.path.join(os.getcwd(),"strain_predictions", "merged_dfs", "tof_indicator", "df_DMD_time_ed2p.csv"))
        strain_p2p = pd.read_csv(os.path.join(os.getcwd(),"strain_predictions", "merged_dfs","tof_indicator", "df_DMD_time_p2p.csv"))
    elif tof_indicator is False and tof_gcn is True:
        strain_ed2p = pd.read_csv(os.path.join(os.getcwd(),"strain_predictions", "merged_dfs","tof_gcn", "df_DMD_time_ed2p.csv"))
        strain_p2p = pd.read_csv(os.path.join(os.getcwd(),"strain_predictions", "merged_dfs","tof_gcn", "df_DMD_time_p2p.csv"))

    target = 'Outcome y/n' # Outcome y/n for deceased patient or not.
    sig_niv = 0.05
    
    if per_aha is True:
        segments_of_interest = list(range(1,17))
        print(segments_of_interest)
        for i in segments_of_interest:
            _ = plot_strain_per_time(strain_p2p[strain_p2p['aha']==i], title="K2K strain - split by LGE class for AHA {}".format(i), method='p2p', hue=target)
            _ = plot_strain_per_time(strain_ed2p[strain_ed2p['aha']==i], title="Composed aligned strain - split by LGE class for AHA {}".format(i), method='p2p', hue=target)
        
        
    else:
        _ = plot_strain_per_time(strain_ed2p,title='Composed aligned Strain - all segments - per cardiac phase',method='comp', hue=target, sig_niv=sig_niv)
        _ = plot_strain_per_time(strain_p2p, title='K2K Strain - all segments - per cardiac phase',method='p2p', hue=target, sig_niv=sig_niv)


interactive(children=(Checkbox(value=True, description='tof_indicator'), Checkbox(value=False, description='to…